### 01 Actividad práctica: Evaluación realista en el Titanic: Riesgos del sobreajuste y la evaluación poco realista

**Objetivo**:

Los estudiantes aprenderán cómo un modelo de Machine Learning puede tener un 
rendimiento sobreestimado cuando el conjunto de prueba no es independiente y 
cómo este tipo de error puede afectar la toma de decisiones en proyectos 
reales. 

**Requisitos:**

- Python (con bibliotecas como pandas, scikit-learn, matplotlib) 
- Jupyter Notebook o entorno similar 
- Conjunto de datos de Titanic disponible en Kaggle Titanic Dataset.

In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split # Se usa para dividir los datos en entrenamiento y test
from sklearn.preprocessing import OneHotEncoder # Se usa para codificar las variables categóricas
from sklearn.compose import ColumnTransformer   # Se usa para transformar las columnas de un dataframe
from sklearn.pipeline import Pipeline   # Se usa para crear un pipeline de transformaciones
from sklearn.impute import SimpleImputer        # Se usa para rellenar los valores faltantes
from sklearn.tree import DecisionTreeClassifier # Se usa para crear un árbol de decisión
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # Se usa para evaluar el modelo

In [134]:
df = pd.read_csv('titanic.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Descripción de la actividad:**

1. Cargar y preprocesar los datos: Cargar el conjunto de datos Titanic 
desde un archivo CSV o Kaggle, preprocesar los datos (tratamiento de 
valores faltantes, codificación de variables categóricas, etc.). 

In [135]:
# Definir las columnas numéricas y categóricas
numeric_features = ['Age', 'Fare']
categorical_features = ['Embarked', 'Sex']


# Tratamiento de valores faltantes
non_empty_categorical = [col for col in categorical_features if df [col].notna().any()]

# Crear tranformador para columnas numéricas
numeric_transformer = SimpleImputer(strategy='mean')

# Crear transformador para columnas categóricas
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
])

# Combinar los transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, non_empty_categorical)
    ]
)

# Seleccionamnos las características (x) y la variable objetivo (y)
x = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived']


# Aplicar el preprocesador a las características
x = preprocessor.fit_transform(x)


# Convertir el array resultante a un Dataframe de pandas
x = pd.DataFrame(x)

# Dividir los datos del conjunto de entrenamiento y prueba
test_size = 0.2
random_state = 42
shuffle = True
stratify = 'mean'

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state, shuffle=shuffle, stratify=y)

# Introducir un conjunto de datos de prueba no independiente
x_test_fake = pd.concat([x_test, x_train.sample(10)])
y_test_fake = pd.concat([y_test, y_train.sample(10)])




2. Entrenar un modelo: Usar un modelo de clasificación (por ejemplo, Árbol 
de Decisión o Regresión Logística) para predecir la supervivencia de los 
pasajeros basándose en características como la clase, el sexo, la edad, 
el puerto de embarque, etc.

In [136]:
# Entrenar el modelo de clasificación

# Entrenar el modelo de árbol de decisión con hiperparámetros especializados
model = DecisionTreeClassifier(max_depth=5, min_samples_split=5, random_state=random_state, min_samples_leaf=2)
model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

3. Evaluación del modelo: 

- Evaluación inicial (métrica con datos de prueba no 
independientes): Evaluar el modelo usando un conjunto de prueba 
que tenga algunos datos duplicados o filtrados del conjunto de 
entrenamiento. 
- Evaluación con datos de prueba verdaderamente independientes: 
Evaluar el modelo en un conjunto de prueba completamente independiente (datos que no se han utilizado en el entrenamiento). 

In [137]:
# Evaluar el modelo en el conjunto de prueba no independiente
y_pred_fake = model.predict(x_test_fake)

# Imprimir las métricas de evaluación
print('Evaluación con conjunto de prueba no independiente')
print('Accuracy:', accuracy_score(y_test_fake, y_pred_fake))
print('Matriz de confusión:\n', confusion_matrix(y_test_fake, y_pred_fake))
print('Reporte de clasificación:\n', classification_report(y_test_fake, y_pred_fake))

Evaluación con conjunto de prueba no independiente
Accuracy: 0.8042328042328042
Matriz de confusión:
 [[103  15]
 [ 22  49]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.82      0.87      0.85       118
           1       0.77      0.69      0.73        71

    accuracy                           0.80       189
   macro avg       0.79      0.78      0.79       189
weighted avg       0.80      0.80      0.80       189



In [140]:
# Evaluar el modelo en el conjunto de prueba independiente
y_pred = model.predict(x_test)

# Imprimir las métricas de evaluación
print('Evaluación con conjunto de prueba independiente')
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Matriz de confusión:\n', confusion_matrix(y_test, y_pred))
print('Reporte de clasificación:\n', classification_report(y_test, y_pred))


Evaluación con conjunto de prueba independiente
Accuracy: 0.8044692737430168
Matriz de confusión:
 [[96 14]
 [21 48]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.82      0.87      0.85       110
           1       0.77      0.70      0.73        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179



4. Observación de las métricas: Los estudiantes compararán las métricas de 
rendimiento (precisión, recall, F1-score, etc.) obtenidas de ambas 
evaluaciones (no independiente e independiente) y observarán cómo se 
inflan las métricas en el primer caso.

5. Conclusión: Los estudiantes documentarán cómo un conjunto de prueba no 
independiente puede dar lugar a una evaluación sobreestimada y cómo esto 
puede afectar la implementación de modelos en la práctica.